# Exloratory Analysis with SQL

In [1]:
import sys
sys.path
sys.path.append('C:\\Users\\clse5199\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python37\\site-packages')

In [2]:
%load_ext sql

In [3]:
import csv, sqlite3
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

Connecting to 'sqlite:///my_data1.db'

In [5]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

In [6]:
%sql create table if not exists SPACEXTABLE as select * from SPACEXTBL where Date is not null

Running query in 'sqlite:///my_data1.db'

++
||
++
++

In [7]:
%sql SELECT COUNT(*) FROM SPACEXTABLE

Running query in 'sqlite:///my_data1.db'

COUNT(*)
101


### Task 1
##### Display the names of the unique launch sites  in the space mission

In [8]:
%%sql 
SELECT DISTINCT LAUNCH_SITE AS 'UNIQUE LAUNCH SITES'
FROM SPACEXTABLE 
ORDER BY LAUNCH_SITE

Running query in 'sqlite:///my_data1.db'

UNIQUE LAUNCH SITES
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 

In [9]:
%%sql 
SELECT *
FROM SPACEXTABLE 
WHERE LAUNCH_SITE LIKE 'CCA%'  
ORDER BY DATE DESC 
LIMIT 5

Running query in 'sqlite:///my_data1.db'

Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2020-11-25,2:13:00,F9 B5 B1049.7,CCAFS SLC-40,"Starlink 15 v1.0, SpaceX CRS-21",15600,LEO,SpaceX,Success,Success
2020-11-05,23:24:23,F9 B5B1062.1,CCAFS SLC-40,"GPS III-04 , Crew-1",4311,MEO,USSF,Success,Success
2020-10-24,15:31:34,F9 B5 B1060.3,CCAFS SLC-40,"Starlink 14 v1.0, GPS III-04",15600,LEO,SpaceX,Success,Success
2020-08-30,23:18:00,F9 B5 B1059.4,CCAFS SLC-40,"SAOCOM 1B, GNOMES 1, Tyvak-0172",3130,SSO,"CONAE, PlanetIQ, SpaceX",Success,Success
2020-08-18,14:31:00,F9 B5 B1049.6,CCAFS SLC-40,"Starlink 10 v1.0, SkySat-19, -20, -21, SAOCOM 1B",15440,LEO,"SpaceX, Planet Labs, PlanetIQ",Success,Success


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [10]:
%%sql 
SELECT SUM(PAYLOAD_MASS__KG_) AS 'TOTAL PAYLOAD MASS BY NASA (CRS)'
FROM SPACEXTABLE 
WHERE CUSTOMER = 'NASA (CRS)'

Running query in 'sqlite:///my_data1.db'

TOTAL PAYLOAD MASS BY NASA (CRS)
45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1

In [11]:
%%sql 
SELECT AVG(PAYLOAD_MASS__KG_) AS 'AVERAGE PAYLOAD MASS BY F9 V1.1'
FROM SPACEXTABLE 
WHERE BOOSTER_VERSION = 'F9 v1.1'

Running query in 'sqlite:///my_data1.db'

AVERAGE PAYLOAD MASS BY F9 V1.1
2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 

In [12]:
%%sql 
SELECT MIN(DATE) AS 'FIRST SUCCESSFUL GROUND PAD LANDING' 
FROM SPACEXTABLE 
WHERE LANDING_OUTCOME LIKE 'Success (ground pad)' 

Running query in 'sqlite:///my_data1.db'

FIRST SUCCESSFUL GROUND PAD LANDING
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [13]:
%%sql
SELECT DISTINCT BOOSTER_VERSION AS 'SUCCESSFUL DRONE SHIP LANDING BOOSTERS'
FROM SPACEXTABLE 
WHERE LANDING_OUTCOME = 'Success (drone ship)'
  AND PAYLOAD_MASS__KG_ > 4000
  AND PAYLOAD_MASS__KG_ < 6000

Running query in 'sqlite:///my_data1.db'

SUCCESSFUL DRONE SHIP LANDING BOOSTERS
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### Task 7




##### List the total number of successful and failure mission outcomes

In [14]:
%sql SELECT COUNT(DISTINCT MISSION_OUTCOME) FROM SPACEXTABLE

Running query in 'sqlite:///my_data1.db'

COUNT(DISTINCT MISSION_OUTCOME)
4


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [15]:
%config SqlMagic.displaylimit = 15

In [16]:
%%sql 
SELECT  DISTINCT BOOSTER_VERSION, 
        PAYLOAD_MASS__KG_  
FROM SPACEXTABLE 
WHERE PAYLOAD_MASS__KG_ IN (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE) 
ORDER BY BOOSTER_VERSION

Running query in 'sqlite:///my_data1.db'

Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1048.5,15600
F9 B5 B1049.4,15600
F9 B5 B1049.5,15600
F9 B5 B1049.7,15600
F9 B5 B1051.3,15600
F9 B5 B1051.4,15600
F9 B5 B1051.6,15600
F9 B5 B1056.4,15600
F9 B5 B1058.3,15600


In [17]:
%config SqlMagic.displaylimit = 10

### Task 9
##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.

In [18]:
%%sql 
SELECT  substr(Date, 6,2) AS MONTH, 
        substr(Date,0,5) AS YEAR,
        Landing_Outcome,
        Booster_Version,
        Launch_Site
FROM SPACEXTABLE
WHERE YEAR = '2015'
  AND LANDING_OUTCOME = 'Failure (drone ship)'

Running query in 'sqlite:///my_data1.db'

MONTH,YEAR,Landing_Outcome,Booster_Version,Launch_Site
01,2015,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,2015,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [19]:
%%sql 
SELECT  LANDING_OUTCOME, 
        COUNT(LANDING_OUTCOME),
        MIN(DATE) AS MIN_DATE,
        MAX(DATE) AS MAX_DATE,
        CAST(substr(Date,0,5) AS DECIMAL) AS YEAR,
        CAST(substr(Date,6,2) AS DECIMAL) AS MONTH,
        CAST(substr(Date,9,2) AS DECIMAL) AS DAY
FROM SPACEXTABLE 
WHERE (YEAR > 2010 OR (YEAR=2010 AND MONTH>6) OR (YEAR=2010 AND MONTH=6 AND DAY>=4))
  AND (YEAR < 2017 OR (YEAR=2017 AND MONTH<3) OR (YEAR=2010 AND MONTH=3 AND DAY<=20))
GROUP BY LANDING_OUTCOME 
ORDER BY COUNT(LANDING_OUTCOME) DESC

Running query in 'sqlite:///my_data1.db'

Landing_Outcome,COUNT(LANDING_OUTCOME),MIN_DATE,MAX_DATE,YEAR,MONTH,DAY
No attempt,9,2012-05-22,2015-04-27,2015,4,27
Success (drone ship),5,2016-04-08,2017-01-14,2017,1,14
Failure (drone ship),5,2015-01-10,2016-06-15,2016,6,15
Success (ground pad),3,2015-12-22,2017-02-19,2017,2,19
Controlled (ocean),3,2014-04-18,2015-02-11,2015,2,11
Uncontrolled (ocean),2,2013-09-29,2014-09-21,2014,9,21
Failure (parachute),2,2010-06-04,2010-12-08,2010,12,8
Precluded (drone ship),1,2015-06-28,2015-06-28,2015,6,28


In [20]:
%%sql
SELECT DISTINCT Launch_Site FROM SPACEXTABLE 

Running query in 'sqlite:///my_data1.db'

Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40
